In [1]:
import pandas as pd
import pickle
from tqdm import tqdm
import os
import re
import random
from tqdm import trange
import operator

In [2]:
def load_id_mapping(filename_list, i):
    id_map_dict = {}
    for filename in filename_list:
        f = open(filename, "r")
        while True:
            line = f.readline()
            if len(line) == 0:
                break
            slots = re.split(r'\t', line)
            old_id = slots[2]
            new_id = slots[i+2]
            new_id = re.split(r'\n', new_id)
            
            id_map_dict[new_id[0]] = old_id 
        f.close()
    return id_map_dict

In [3]:
RETURN_NUM = 100
STEP = 20
key=[] 
value=[]
for number in range(RETURN_NUM):
    key.append(number)
    i = number / STEP + 1
    value.append(1/i)
## Weight
dic=dict(zip(key,value)) 

In [15]:
def load_result(filename_list):
    predict_dict = {}
    gt_dict = {}
    for i in range(len(filename_list)):
        f = open(filename_list[i], "r")
        idx = re.findall(r"\d+",filename_list[i])
        id_map_dict = load_id_mapping(['../Data_process/Trivia_dataset/trivia_title_cont.tsv'], int(idx[0]))
            
        for line in f.readlines():
            query, predict_id, gt_id, rank = line[:-1].split("\t")
            
            gt_id = list(gt_id.split(","))
            for i in range(len(gt_id)):
                gt_id[i] = id_map_dict[str(gt_id[i])]
            
            pred_id = [[], []]
            predict_id = list(predict_id.split(","))
            for i in range(RETURN_NUM):
                pred_id[0].append(id_map_dict[predict_id[i]])
                pred_id[1].append(dic[i])
            gt_dict[query] = gt_id
            if query not in predict_dict:
                predict_dict[query] = []
                predict_dict[query].append(pred_id)
            else:
                predict_dict[query][0][0].extend(pred_id[0])
                predict_dict[query][0][1].extend(pred_id[1])
            # predict_dict[query] = predict_dict[query][0:previous_k] + predict_dict[query][max(0,len(predict_dict[query])-k):len(predict_dict[query])]
        f.close()
    return predict_dict, gt_dict

## add your result path
## You can download our prediction results as well.
predict_dict, gt_dict = load_result([
                                    './logs/trivia_res1_1.tsv',
                                     './logs/trivia_res1_2.tsv',
                                     './logs/trivia_res2.tsv',
                                     './logs/trivia_res3.tsv',
                                     './logs/trivia_res4.tsv',
                                     './logs/trivia_res5.tsv'
                                    ])

In [16]:
RECALL_NUM = 1
for key,value in predict_dict.items():
        dic_cnt = {}
        for i in range(len(predict_dict[key][0][0])):
            if predict_dict[key][0][0][i] not in dic_cnt:
                dic_cnt[predict_dict[key][0][0][i]] = predict_dict[key][0][1][i]
            else:
                dic_cnt[predict_dict[key][0][0][i]] += predict_dict[key][0][1][i]
        dic_cnt = sorted(dic_cnt.items(),key=operator.itemgetter(1),reverse=True)
        pred_ans = []
        for i in dic_cnt[:RECALL_NUM]:
            pred_ans.append(i[0])
        predict_dict[key] = pred_ans

In [6]:
total = 0
for q in predict_dict:
    right = 0
    for p in gt_dict[q]:
        if p in predict_dict[q]:
            right = 1
    recall = right
    total += recall
recall_avg = total / len(predict_dict)
print(f"recall@{RECALL_NUM}: {recall_avg}")

recall@5: 0.9456412825651302


In [11]:
total = 0
for q in predict_dict:
    right = 0
    for p in gt_dict[q]:
        if p in predict_dict[q]:
            right = 1
    recall = right 
    total += recall
recall_avg = total / len(predict_dict)
print(f"recall@{RECALL_NUM}: {recall_avg}")

recall@20: 0.9720691382765531


In [14]:
total = 0
for q in predict_dict:
    right = 0
    for p in gt_dict[q]:
        if p in predict_dict[q]:
            right = 1
    recall = right 
    total += recall
recall_avg = total / len(predict_dict)
print(f"recall@{RECALL_NUM}: {recall_avg}")

recall@100: 0.9829659318637275


In [17]:
total = 0
for q in predict_dict:
    right = 0
    for p in gt_dict[q]:
        if p in predict_dict[q]:
            right = 1
    recall = right 
    total += recall
recall_avg = total / len(predict_dict)
print(f"R-Precision: {recall_avg}")

R-Precision: 0.8186372745490982
